In [1]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import numpy as np

load_dotenv()

# Get the API key from the environment
API_KEY = os.getenv("STEAM_API_KEY")
MARKET_API_KEY = os.getenv("STEAM_MARKET_API_KEY")

In [2]:
steam_store = "https://store.steampowered.com"
# /appreviews/10500?
base_params = {
    "json": 1,
    "language": "english",
    "filter": "recent",
    "num_per_page": 100,
    "cursor": "*",
}

In [3]:
steam_market = 'https://api.steamapis.com'
app_list_url = f"{steam_market}/market/apps?"
market_params = {
    "api_key": MARKET_API_KEY
}

In [4]:
all_games = requests.get(app_list_url, params=market_params).json()

In [8]:
all_games_df = pd.DataFrame(all_games)

In [3]:
all_games_df.head()

,appID,name,is_free,price_overview
0,10500,Total War: EMPIRE – Definitive Edition,False,"{'final_formatted': '$24.99', 'initial_formatt..."
1,391070,Stellar 2D,False,"{'final_formatted': '$0.99', 'initial_formatte..."
2,576670,Sky Is Arrows,False,"{'final_formatted': '$9.99', 'initial_formatte..."
3,256410,Might & Magic: Duel of Champions,True,NaN
4,546330,SPACE MOUSE 35th Anniversary edition,False,"{'final_formatted': '$4.99', 'initial_formatte..."


In [10]:
all_games_df.to_csv("data/steam/all_games.csv", index=False)

In [4]:
all_games_df = pd.read_csv("data/steam/all_games.csv")

In [5]:
all_games_df.head()

,appID,name,is_free,price_overview
0,10500,Total War: EMPIRE – Definitive Edition,False,"{'final_formatted': '$24.99', 'initial_formatt..."
1,391070,Stellar 2D,False,"{'final_formatted': '$0.99', 'initial_formatte..."
2,576670,Sky Is Arrows,False,"{'final_formatted': '$9.99', 'initial_formatte..."
3,256410,Might & Magic: Duel of Champions,True,NaN
4,546330,SPACE MOUSE 35th Anniversary edition,False,"{'final_formatted': '$4.99', 'initial_formatte..."


In [24]:
import urllib

reviews_from_api = []

for app_id in all_games_df["appID"]:
    cursor = '*'
    reviews_url = f"{steam_store}/appreviews/{app_id}?"
    past_cursors = []

    positive = False

    while cursor:
        params = base_params.copy()
        params["cursor"] = cursor
        params["review_type"] = 'positive' if positive else "negative"
        url = f"{reviews_url}"\
        + f"json=1&language=english&filter=recent&num_per_page=100&"\
        + f"cursor={cursor}&review_type={'positive' if positive else 'negative'}"
        # print(url)
        res = requests.get(url)
        reviews = res.json()
        cursor = reviews.get("cursor")

        if cursor:
            cursor = urllib.parse.quote(cursor)
        if cursor in past_cursors:
            print("Duplicate cursor found, breaking")
            print(app_id)
            print(cursor)
            print(past_cursors)
            if positive:
                break
            positive = True
            cursor = '*'
            past_cursors = []
            continue
        if not cursor:
            print("No cursor found, breaking")
            if positive:
                break
            positive = True
            cursor = '*'
            past_cursors = []
            continue
        past_cursors.append(cursor)

        if cursor:
            for review in reviews["reviews"]:
                author = review["author"]["steamid"]

                reviews_from_api.append({
                    "app_id": app_id,
                    "author": author,
                    "recommended": positive
                })
        else:
            break

# sort reviews by review_weighted
reviews_df = pd.DataFrame(reviews_from_api)

# set max number of rows to none
pd.set_option('display.max_rows', None)

reviews_df.tail()


Duplicate cursor found, breaking
10500
AoJ4s9yN7awCfKT3Aw%3D%3D
['AoJ4wvf0sosDcP7htwQ%3D', 'AoJwqtLR2oQDdLnj5AM%3D', 'AoJwgLCC6PwCeLvLhAM%3D', 'AoJws/K5nPUCcMTCqAI%3D', 'AoJ4mIWpzO8CeY%2B96wE%3D', 'AoJ4pu%2Bz2ecCdNSxtQE%3D', 'AoJ4waeY%2BOICeZ3qngE%3D', 'AoJ4h8La5dkCf4OocA%3D%3D', 'AoJwxt/QjtECdO%2BHSQ%3D%3D', 'AoJwr/qjx8oCfKCKNA%3D%3D', 'AoJw%2BdbN57gCfM2MDw%3D%3D', 'AoJ4s9yN7awCfKT3Aw%3D%3D']
Duplicate cursor found, breaking
10500
AoJ4zcP4tKwCetuXBA%3D%3D
['AoJ4nejy2I8DeJe9%2BQQ%3D', 'AoJw1LnX144DcpeE6wQ%3D', 'AoJwmdvH540Df6Wq3gQ%3D', 'AoJ4wquFlI0Dcp/O1AQ%3D', 'AoJw0qHwt4wDfK6cygQ%3D', 'AoJw9aKZ9osDcfeGvgQ%3D', 'AoJw%2BrLriosDdqurtAQ%3D', 'AoJ4oprls4oDe5n3qwQ%3D', 'AoJ4zaDWwokDee2hoAQ%3D', 'AoJ4z7ad6IgDcvKalwQ%3D', 'AoJwxL7N9IcDdojXjQQ%3D', 'AoJ4p8DnhIcDfILMgwQ%3D', 'AoJw6MfV9YUDcLXs9QM%3D', 'AoJ4y6KroIUDdqPH7AM%3D', 'AoJw38Omw4QDffWi3wM%3D', 'AoJ4hMT1zYMDeo%2Bb1gM%3D', 'AoJ44c270YIDerWyzAM%3D', 'AoJ4kOuN44EDf77fwgM%3D', 'AoJw6uXwmIEDc5a2vAM%3D', 'AoJ4zszks4ADftzxtAM%3D', 'AoJw5syj2/8

In [21]:
reviews_df.tail()

,app_id,author,recommended
10584,10500,76561197968358961,True
10585,10500,76561198023760188,True
10586,10500,76561198023472805,True
10587,10500,76561198008870367,True
10588,10500,76561197986631670,True


In [19]:
display(len(reviews_df))
display(reviews_df.duplicated().sum())
reviews_df.drop_duplicates(inplace=True)
display(reviews_df.duplicated().sum())

display(len(reviews_df))

10589

0

0

10589

In [22]:
print(reviews_df.head())
print(reviews_df.tail())

   app_id             author  recommended
0   10500  76561198140439494        False
1   10500  76561198001149578        False
2   10500  76561199667392107        False
3   10500  76561198024411821        False
4   10500  76561198297284962        False
       app_id             author  recommended
10584   10500  76561197968358961         True
10585   10500  76561198023760188         True
10586   10500  76561198023472805         True
10587   10500  76561198008870367         True
10588   10500  76561197986631670         True


In [13]:
params = {
    "key": API_KEY,
    "format": "json",
    "page": 2
}
app_list = requests.get(app_list_url, params=base_params).json()
app_list


{'applist': {'apps': [{'appid': 1941401, 'name': ''},
   {'appid': 1897482, 'name': ''},
   {'appid': 2112761, 'name': ''},
   {'appid': 2016512, 'name': ''},
   {'appid': 1820332, 'name': ''},
   {'appid': 1360782, 'name': ''},
   {'appid': 662172, 'name': ''},
   {'appid': 216938, 'name': 'Pieterw test app76 ( 216938 )'},
   {'appid': 660010, 'name': 'test2'},
   {'appid': 660130, 'name': 'test3'},
   {'appid': 1118314, 'name': ''},
   {'appid': 1275822, 'name': ''},
   {'appid': 1343832, 'name': ''},
   {'appid': 1828741, 'name': ''},
   {'appid': 1927051, 'name': ''},
   {'appid': 1496152, 'name': ''},
   {'appid': 1983382, 'name': ''},
   {'appid': 1808781, 'name': ''},
   {'appid': 1977312, 'name': ''},
   {'appid': 1700632, 'name': ''},
   {'appid': 1829051, 'name': ''},
   {'appid': 1567401, 'name': ''},
   {'appid': 2092072, 'name': ''},
   {'appid': 2119422, 'name': ''},
   {'appid': 596501, 'name': ''},
   {'appid': 2156011, 'name': ''},
   {'appid': 2177061, 'name': ''},
  

In [14]:
review_url = 'http://store.steampowered.com/appreviews/440?json=1'

In [9]:
requests.get(review_url, params=base_params).json()

{'success': 1,
 'query_summary': {'num_reviews': 20,
  'review_score': 5,
  'review_score_desc': 'Mixed',
  'total_positive': 12194,
  'total_negative': 11291,
  'total_reviews': 23485},
 'reviews': [{'recommendationid': '166633634',
   'author': {'steamid': '76561199102102581',
    'num_games_owned': 52,
    'num_reviews': 1,
    'playtime_forever': 102902,
    'playtime_last_two_weeks': 1018,
    'playtime_at_review': 99999,
    'last_played': 1719527834},
   'language': 'english',
   'review': "TF2 is an amazing game, yet its neglected by Valve. I'm gunna do my part for this awesome community and fight for a better TF2, one without bots and a game that won't go ignored anymore.\n\n#fixtf2",
   'timestamp_created': 1717465234,
   'timestamp_updated': 1717465234,
   'voted_up': False,
   'votes_up': 161,
   'votes_funny': 0,
   'weighted_vote_score': '0.842052221298217773',
   'comment_count': 0,
   'steam_purchase': True,
   'received_for_free': False,
   'written_during_early_access